In [1]:
!nvidia-smi

Sun Jun 11 17:53:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#install missing dependancies
!pip install transformers accelerate


#Library used for fine tuning

from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments

# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 
import datetime


# HuggingFace Libarary
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer, BertConfig, BertTokenizer
from transformers import EarlyStoppingCallback



#All tested models 
bert_cased="bert-base-cased"
bert="bert-base-uncased"
HateBert="GroNLP/hateBERT"
DistilBert="distilbert-base-uncased"
RoBERTa="roberta-base" 
HateRoBERTa="facebook/roberta-hate-speech-dynabench-r4-target"


# Model HyperParameters
current_model=RoBERTa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
gold='/content/drive/MyDrive/BSP6/init_dataset_gold'
gold_pre_processed = '/content/drive/MyDrive/BSP6/init_dataset_gold_preprocessed' 
gold_oversampeld ='/content/drive/MyDrive/BSP6/init_dataset_gold_silver_balance'
gold_undersampeld ='/content/drive/MyDrive/BSP6/init_dataset_gold_balance'
gold_oversample_pre='/content/drive/MyDrive/BSP6/init_dataset_gold_preprocessed_oversample'


current_dataset=gold_oversample_pre
os.chdir(current_dataset) 
now = str(datetime.datetime.now())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data = pd.read_json('train.json', lines=True)
val_data = pd.read_json('val.json', lines=True)
test_data = pd.read_json('test.json', lines=True)

In [5]:
# load pre-trained HateBert
if(current_model=="roberta-base"):
  model = AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=3)
  tokenizer = AutoTokenizer.from_pretrained('roberta-base')
elif(current_model=="facebook/roberta-hate-speech-dynabench-r4-target"):
  tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
  model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target",num_labels=3, ignore_mismatched_sizes=True,hidden_dropout_prob=0.3,attention_probs_dropout_prob=0.3)
else:
  model = BertForSequenceClassification.from_pretrained(current_model,num_labels=3 )
  tokenizer = BertTokenizer.from_pretrained(current_model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should pr

In [6]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [7]:
#Train Data
#max length var
max_length_X=225
max_length_V=225
max_length_T=225

#truncation var
tru = True

X = []
for context, target in zip(train_data.context, train_data.target):
  X.append(tokenizer(context, target, padding='max_length', truncation=tru, max_length=max_length_X))
y_train = list(train_data.label)
print(X)
X_train_tokenized = X


#val Data
V = []
for context, target in zip(val_data.context, val_data.target):
   V.append(tokenizer( context,target, padding='max_length', truncation=tru, max_length=max_length_V))
y_val = list(val_data.label)
V_val_tokenized = V

#test Data
T = []
for context, target in zip(test_data.context, test_data.target):
   T.append(tokenizer(context, target, padding='max_length', truncation=tru, max_length=max_length_T))
y_test = list(test_data.label)
T_test_tokenized = T





IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#print the token
print(tokenizer.decode(X_train_tokenized[0].input_ids))
print(X_train_tokenized[0])
print(len(X_train_tokenized[0].input_ids))
print(y_train[0])


print(tokenizer.decode(X_train_tokenized[1].input_ids))
print(X_train_tokenized[1])
print(len(X_train_tokenized[1].input_ids))
print(y_train[1])


<s>The UK is fucked.</s></s>>The ~~UK~~ world is fucked  FTFY</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [9]:
from torch.utils.data import DataLoader, RandomSampler

# Create torch dataset
#need a dictionary
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids':torch.tensor(self.encodings[idx].input_ids),
                'attention_mask':torch.tensor(self.encodings[idx].attention_mask),
                'labels':torch.tensor(self.labels[idx])}
        
        return item

    def __len__(self):
        return len(self.encodings)

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(V_val_tokenized, y_val)
test_dataset = Dataset(T_test_tokenized, y_test)


In [10]:
train_dataset[0]

{'input_ids': tensor([    0,   133,   987,    16, 42647,     4,     2,     2, 15698,   133,
          1437, 48256, 10494, 48256,   232,    16, 42647,  1437,   274, 20249,
           975,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
#importing confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
f = open(current_dataset+"/Report/output"+now+".txt","w")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    confusion = confusion_matrix(labels, preds)
    print('Confusion Matrix\n')
    print(confusion)
    print('\nClassification Report\n')
    print(classification_report(labels, preds, target_names=['Hate', 'Neutral', 'Counter']))
    f.write('Confusion Matrix\n')
    f.write(str(confusion))
    f.write('\nClassification Report\n')
    f.write(str(classification_report(labels, preds, target_names=['Hate', 'Neutral', 'Counter'])))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [12]:

# Define Trainer
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,               # total number of training epochs
    per_device_train_batch_size=32,   # batch size per device during training
    per_device_eval_batch_size=32,    # batch size for evaluation
    learning_rate = 2e-5,
    evaluation_strategy="steps",
    logging_steps = 50,
    weight_decay=0.1,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    seed=42
)            



trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)


In [13]:


f.write("model"+ current_model)
f.write("dataset: "+ current_dataset)
f.write("\n")
f.write("structure of the model: \n")
f.write(str(trainer.model))
f.write("\n")
f.write("Tokenizer max length train:"+str(max_length_X))
f.write("Tokenizer max length val:"+str(max_length_V))
f.write("Tokenizer max length test:"+str(max_length_T))
f.write("\n")
f.write(str(trainer.args))
f.write("\n")

1

In [14]:
trainer.train()
f.write(str(trainer.state.log_history))
eval=trainer.evaluate(test_dataset)
f.write(str(eval))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.096900,1.087224,0.380342,0.144660,0.380342,0.209600
100,1.071800,1.051747,0.436966,0.377486,0.436966,0.350568
150,0.999600,1.002014,0.530983,0.531134,0.530983,0.512277
200,0.928900,1.004758,0.518162,0.523120,0.518162,0.515887
250,0.912700,0.979216,0.511752,0.511514,0.511752,0.511617
300,0.829000,1.006334,0.553419,0.557830,0.553419,0.554259
350,0.811200,1.001195,0.524573,0.544561,0.524573,0.526223
400,0.788100,0.998448,0.538462,0.539737,0.538462,0.534633
450,0.680300,1.039732,0.560897,0.556378,0.560897,0.557027
500,0.649600,1.071253,0.568376,0.561078,0.568376,0.555155


Confusion Matrix

[[  0 331   0]
 [  0 356   0]
 [  0 249   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.00      0.00      0.00       331
     Neutral       0.38      1.00      0.55       356
     Counter       0.00      0.00      0.00       249

    accuracy                           0.38       936
   macro avg       0.13      0.33      0.18       936
weighted avg       0.14      0.38      0.21       936



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Confusion Matrix

[[300  31   0]
 [247 109   0]
 [215  34   0]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.39      0.91      0.55       331
     Neutral       0.63      0.31      0.41       356
     Counter       0.00      0.00      0.00       249

    accuracy                           0.44       936
   macro avg       0.34      0.40      0.32       936
weighted avg       0.38      0.44      0.35       936



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Confusion Matrix

[[251  59  21]
 [121 184  51]
 [125  62  62]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.51      0.76      0.61       331
     Neutral       0.60      0.52      0.56       356
     Counter       0.46      0.25      0.32       249

    accuracy                           0.53       936
   macro avg       0.52      0.51      0.50       936
weighted avg       0.53      0.53      0.51       936

Confusion Matrix

[[152  92  87]
 [ 55 232  69]
 [ 49  99 101]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.59      0.46      0.52       331
     Neutral       0.55      0.65      0.60       356
     Counter       0.39      0.41      0.40       249

    accuracy                           0.52       936
   macro avg       0.51      0.51      0.50       936
weighted avg       0.52      0.52      0.52       936

Confusion Matrix

[[182  72  77]
 [ 81 201  74]
 [ 73  80  96]

Confusion Matrix

[[116  40  28]
 [ 65 263  33]
 [ 25  72  71]]

Classification Report

              precision    recall  f1-score   support

        Hate       0.56      0.63      0.59       184
     Neutral       0.70      0.73      0.71       361
     Counter       0.54      0.42      0.47       168

    accuracy                           0.63       713
   macro avg       0.60      0.59      0.59       713
weighted avg       0.63      0.63      0.63       713



278

In [15]:
eval

{'eval_loss': 0.9269415736198425,
 'eval_accuracy': 0.6311360448807855,
 'eval_precision': 0.627148134898186,
 'eval_recall': 0.6311360448807855,
 'eval_f1': 0.6268915748458709,
 'eval_runtime': 9.5147,
 'eval_samples_per_second': 74.937,
 'eval_steps_per_second': 2.417,
 'epoch': 5.0}

In [16]:
f.close()